# Portfolio Optimization - 
Based on https://examples.pyviz.org/portfolio_optimizer/portfolio.html#portfolio-optimizer-gallery-portfolio

Optimizes the Sharpe Ratio of a portfolio based on the risk profile of the user.
1. Import data
2. Calculate returns
3. Determine constraints base on risk assesment
4. Optimizes portfolio for optimal asset allocation to maximize sharpe ratio

In [251]:
import numpy as np
import pandas as pd
import hvplot.pandas  
from scipy.optimize import minimize, Bounds

In [253]:
portfolio_prices = pd.read_csv('./daily_prices_input_portfolio_optimization.csv', index_col='Date', parse_dates=True)

In [254]:
portfolio_prices.head()

,AGG,SPY,AAPL,CISCO
Date,,,,
2012-01-03,160.830881,179.03,53.063218,15.752778
2012-01-04,160.174781,177.51,53.348386,16.057180
2012-01-05,159.415086,177.61,53.940658,15.997991
2012-01-06,157.584912,182.61,54.504543,15.938801
2012-01-09,156.764787,178.56,54.418089,16.040268


In [259]:
portfolio_daily_returns = portfolio_prices.pct_change(1).dropna()
portfolio_daily_returns.head()

,AGG,SPY,AAPL,CISCO
Date,,,,
2012-01-04,-0.004079,-0.008490,0.005374,0.019324
2012-01-05,-0.004743,0.000563,0.011102,-0.003686
2012-01-06,-0.011481,0.028152,0.010454,-0.003700
2012-01-09,-0.005204,-0.022178,-0.001586,0.006366
2012-01-10,-0.001542,0.004368,0.003580,-0.007380


In [260]:
# Calculates correlation of securities in the client_portfolio_prices
portfolio_prices.pct_change(1).corr()

,AGG,SPY,AAPL,CISCO
AGG,1.000000,0.258492,0.297498,0.424672
SPY,0.258492,1.000000,0.235487,0.284470
AAPL,0.297498,0.235487,1.000000,0.301990
CISCO,0.424672,0.284470,0.301990,1.000000


## Switching to log returns

We will switch over to using log returns instead of arithmetic returns, because they are more normal distributed.
Log returns are similar to regular returns for small values, but reduce the size of outliers.

For a full analysis of why we use log returns, check [this article](https://quantivity.wordpress.com/2011/02/21/why-log-returns/).


In [261]:
log_daily_returns = np.log(portfolio_prices/portfolio_prices.shift(1)).dropna()
log_daily_returns.head()

,AGG,SPY,AAPL,CISCO
Date,,,,
2012-01-04,-0.004088,-0.008526,0.005360,0.019139
2012-01-05,-0.004754,0.000563,0.011041,-0.003693
2012-01-06,-0.011547,0.027763,0.010400,-0.003707
2012-01-09,-0.005218,-0.022428,-0.001587,0.006346
2012-01-10,-0.001543,0.004359,0.003574,-0.007407


In [262]:
log_daily_returns.hvplot.hist(bins=100, subplots=True, width=400, grid=True).cols(2)

:NdLayout   [Default]
   :Histogram   [CISCO]   (CISCO_count)

In [263]:
log_daily_returns.describe().T

,count,mean,std,min,25%,50%,75%,max
AGG,1257.0,0.000011,0.011819,-0.086419,-0.005873,0.000049,0.006477,0.049130
SPY,1257.0,0.001139,0.019362,-0.116503,-0.008534,0.000563,0.011407,0.146225
AAPL,1257.0,0.000614,0.016466,-0.131875,-0.007358,0.000455,0.009724,0.085022
CISCO,1257.0,0.000497,0.014279,-0.116091,-0.006240,0.000213,0.007634,0.118862


In [265]:
# Annualized returns
log_daily_returns.mean() * 252

AGG      0.002788
SPY      0.287153
AAPL     0.154803
CISCO    0.125291
dtype: float64

In [266]:
# Compute daily covariance matrix
log_daily_returns.cov()

,AGG,SPY,AAPL,CISCO
AGG,0.000140,0.000059,0.000057,0.000072
SPY,0.000059,0.000375,0.000075,0.000079
AAPL,0.000057,0.000075,0.000271,0.000071
CISCO,0.000072,0.000079,0.000071,0.000204


In [248]:
# Annualized covariance matrix by multiplying by trading days in a year
log_daily_returns.cov()*252 

,AGG,SPY,AAPL,CISCO
AGG,0.035203,0.014939,0.014464,0.018029
SPY,0.014939,0.094470,0.018986,0.019956
AAPL,0.014464,0.018986,0.068326,0.017854
CISCO,0.018029,0.019956,0.017854,0.051381


# Portfolio Optimization and Recommendation
Level of risk:
1. Aggressive 
2. Moderate
3. Conservative


In [328]:
# set level of risk according to risk profile
level_of_risk=3

### Functionalize Return and SR operations

In [309]:
def get_annual_expected_return_volatility_and_sharpe_ratio(portfolio_weights):
    """
    Takes in portfolio_weights, returns array or annualized expected return, volatility, sharpe ratio
    """
    portfolio_weights = np.array(portfolio_weights)
    
    #Annual expected return
    expected_annual_return = np.sum(log_daily_returns.mean() * portfolio_weights) * 252
    
    # Annual Portfolio Variance = w.T * Covariance * w    (where "*" represents here the matrix multiplication) OBS: Var_annual = Var_daily * 252
    # np.dot function does a matrix multiplication. Another alternative is np.matmul
    portfolio_variance = np.dot(    portfolio_weights.T,     np.dot(   log_daily_returns.cov() * 252, portfolio_weights   ))
    portfolio_volatility = np.sqrt(portfolio_variance)

    #Sharpe Ratio (annual) (Assumed risk_free_rate=0)
    risk_free_rate_annualized=0
    sharpe_ratio = (expected_annual_return - risk_free_rate_annualized)/portfolio_volatility
    
    return np.array([expected_annual_return,portfolio_volatility,sharpe_ratio]) 

To fully understand all the parameters, check out:
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

In [94]:
#help(minimize)

Optimization works as a minimization function, since we actually want to maximize the Sharpe Ratio, we will need to turn it negative so we can minimize the negative sharpe (same as maximizing the postive sharpe)

In [294]:
# Function to be optimized 
def negative_sharpe_ratio(portfolio_weights):
    return  get_annual_expected_return_volatility_and_sharpe_ratio(portfolio_weights)[2] * -1

In [329]:
# Required arguments for optimization constaints
if level_of_risk==1:
    [equity_lower_limit,equity_upper_limit]  =[0.75, 0.9]

elif level_of_risk==2:
    [equity_lower_limit,equity_upper_limit]  =[0.40, 0.6]

elif level_of_risk==3:
    [equity_lower_limit,equity_upper_limit]  =[0 , 0.2]

else: [equity_lower_limit,equity_upper_limit]=[0 , 1.0]




In [318]:
# Contraints: weight sum must be equal to 1 
def check_sum_of_portfolio_weights_is_one(portfolio_weights):
    '''
    Returns 0 if sum of portfolio_weights is 1.0
    '''
    return np.sum(portfolio_weights) - 1

In [297]:
# Constraint: equity allocation is less than maximum limit (inequality constraint, upper limit - equity allocation >0)
def check_sum_of_portfolio_equity_weights_is_less_than_upper_bound(portfolio_weights, asset_class_securities, equity_upper_limit):
    """
    Makes sure allocation in equity is less than the maximum allowed
    """
    portfolio_equity_allocation=sum(portfolio_weights * asset_class_securities['Equity'])
    return  equity_upper_limit - portfolio_equity_allocation

In [298]:
# Constraint: equity allocation is more than minimum limit (inequality constraint, equity allocation - lower_limit >0)
def check_sum_of_portfolio_equity_weights_is_more_than_lower_bound(portfolio_weights,asset_class_securities , equity_lower_limit):
    """
    Makes sure allocation in equity is more than the minimum allowed
    """
    portfolio_equity_allocation=sum(portfolio_weights * asset_class_securities['Equity'])
    return portfolio_equity_allocation - equity_lower_limit

In [319]:
# By convention of minimize function it should be a function that returns zero for constraints. 
# Constraint type: 'eq' for equality, 'ineq' for inequality. 
# 'fun': The function defining the constraint: sum(portfolio_weights)=1  ==> sum(portfolio_weights) - 1 = 0
#                                                                       upper limit - equity allocation >0
#                                                                       equity allocation - lower_limit >0


cons = [{'type':'eq','fun'  :check_sum_of_portfolio_weights_is_one},
        {'type':'ineq','fun':check_sum_of_portfolio_equity_weights_is_less_than_upper_bound,'args':(asset_class_securities, equity_upper_limit)},
        {'type':'ineq','fun':check_sum_of_portfolio_equity_weights_is_more_than_lower_bound,'args':(asset_class_securities, equity_lower_limit)}]



In [320]:
#Number of instruments and asset classes
n=len(portfolio_prices.columns)
asset_class_securities=pd.read_csv('asset_classes_input_portfolio_optimization.csv')
asset_class_securities=asset_class_securities.set_index('Symbol')
asset_class_securities


,Equity,Fixed_Income,Other
Symbol,,,
AGG,0,1,0
SPY,1,0,0
AAPL,1,0,0
CISCO,1,0,0


In [301]:
# Setting bounds for the weights of each security
lb=[0]*n
ub=[1]*n
        
bounds=Bounds(lb, ub)
bounds

Bounds([0, 0, 0, 0], [1, 1, 1, 1])

In [302]:
# Initial Guess (equal distribution)
# n: number of stocks in the portfolio to optimize
initial_guess = [1/n]*n
initial_guess

[0.25, 0.25, 0.25, 0.25]

In [322]:
# Sequential Least SQuares Programming (SLSQP) - Similar to Least Square optimization, but it uses less resources. Approximate using paraboles.
optimization_results = minimize(negative_sharpe_ratio, initial_guess, method='SLSQP', bounds=bounds, constraints=cons)

In [323]:
# Optimization results
optimization_results

     fun: -1.030716870219052
     jac: array([ 3.39921713e-01, -4.45097685e-05,  5.64157963e-05,  4.18275595e-05])
 message: 'Optimization terminated successfully'
    nfev: 35
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([2.05998413e-17, 5.29520411e-01, 2.66289775e-01, 2.04189813e-01])

In [324]:
# Optimal weights
optimal_portfolio_weights = optimization_results.x

portfolio_position_names=pd.Series(portfolio_prices.columns)
portfolio_position_names

optimal_portfolio_weights=pd.Series(optimal_portfolio_weights)

optimal_portfolio_weights=pd.concat([portfolio_position_names,optimal_portfolio_weights], axis=1)
optimal_portfolio_weights=optimal_portfolio_weights.set_index(0)
optimal_portfolio_weights

,1
0,
AGG,2.059984e-17
SPY,5.295204e-01
AAPL,2.662898e-01
CISCO,2.041898e-01


In [325]:
# Return, volatility, and Sharpe ratio on optimal value (Max Sharpe ratio point)
[optimal_expected_return, optimal_volatility, maximum_Sharpe_ratio]= get_annual_expected_return_volatility_and_sharpe_ratio(optimization_results.x)

print( f"Optimal Annual Expected Return: {optimal_expected_return:.2f}"  )
print( f"Optimal Annual Volatility:      {optimal_volatility:.2f}") 
print( f"Maximum Annual Sharpe ratio:    {maximum_Sharpe_ratio:.2f}")


Optimal Annual Expected Return: 0.22
Optimal Annual Volatility:      0.21
Maximum Annual Sharpe ratio:    1.03


In [327]:
# Return of results
# ORIGINAL PORTFOLIO
original_client_portfolio_weights = pd.read_csv('./original_client_portfolio_weights.csv')
original_client_portfolio_weights=original_client_portfolio_weights.iloc[:,1]

[original_expected_return, original_volatility, original_Sharpe_ratio]=get_annual_expected_return_volatility_and_sharpe_ratio(original_client_portfolio_weights)

print( f"Original Annual Expected Return:  {original_expected_return:.2f}"  )
print( f"Original Annual Volatility:       {original_volatility:.2f}") 
print( f"Original Annual Sharpe ratio:     {original_Sharpe_ratio:.2f}")
original_client_portfolio_weights

Original Annual Expected Return:  0.19
Original Annual Volatility:       0.19
Original Annual Sharpe ratio:     0.98


0    0.00
1    0.32
2    0.27
3    0.41
Name: Original allocation, dtype: float64